In [ ]:
#To do:  Robust comments, sensitivity analysis, hyperparameter tuning, regression forecasting, traditional ML models for comparison

In [5]:
import pandas as pd
import numpy as np
import glob
import os
import re
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing, model_selection
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import svm, datasets
from sklearn.model_selection import cross_val_score

In [6]:
df = pd.read_csv('woodbine_horses.csv')
df.shape

(4370, 27)

In [7]:
print(df.columns)

Index(['track', 'card_id', 'race_date', 'breed', 'name', 'weight', 'age',
       'sex', 'meds_and_equip', 'jockey', 'dollar_odds', 'program_number',
       'post_position', 'claim_price', 'start_position', 'finish_position',
       'finish_time', 'speed_rating', 'trainer', 'owner', 'win_payoff',
       'place_payoff', 'show_payoff', 'weather', 'distance', 'race_type',
       'track_conditions'],
      dtype='object')


In [8]:
#Exploratory analysis
df.head()

,track,card_id,race_date,breed,name,weight,age,sex,meds_and_equip,jockey,...,speed_rating,trainer,owner,win_payoff,place_payoff,show_payoff,weather,distance,race_type,track_conditions
0,WO,D,2015-06-20 00:00:00,TB,Madam Premier,119,4,M,Lb,S Ryan,...,44.0,O'Keefe Thomas,Emily Anne Walker,0.0,0.0,0.0,Cloudy,900.0,Claiming,FT
1,WO,D,2015-07-19 00:00:00,TB,Notion Sonet,118,3,C,L,L Contreras,...,96.0,Carroll Josie,Ivan Dalos,0.0,0.0,0.0,Cloudy,650.0,Allowance,FT
2,WO,D,2015-07-25 00:00:00,TB,Big Red Bugsy,121,5,G,L,S R Bahen,...,83.0,Fournier Mark,"Carruthers, Carruthers, Carruthers, Carruthers...",0.0,0.0,0.0,Cloudy,700.0,Claiming,FT
3,WO,D,2015-08-14 00:00:00,TB,Lillz Pearl,121,5,G,Lb,J McAleney,...,88.0,Singh Ricky,Ricky Singh,0.0,5.7,3.3,Clear,500.0,Allowance,FT
4,WO,D,2015-08-29 00:00:00,TB,Sweet Rose,121,4,F,NaN,L Contreras,...,86.0,Attard Sid C,Stronach Stables,0.0,0.0,3.1,Cloudy,500.0,Allowance,FT


In [9]:
#There's only 1 value for the track feature, making it unnecessary
df['track'].value_counts()

WO     4370
Name: track, dtype: int64

In [10]:
#There's only 1 value for the track feature, making it unnecessary
df['card_id'].value_counts()

D    4370
Name: card_id, dtype: int64

In [7]:
df['race_date'].value_counts()

2016-10-16 00:00:00    106
2016-09-25 00:00:00     92
2016-10-29 00:00:00     89
2016-10-09 00:00:00     89
2016-09-17 00:00:00     88
                      ... 
2016-05-19 00:00:00      1
2016-05-15 00:00:00      1
2016-05-14 00:00:00      1
2016-05-13 00:00:00      1
2017-04-05 00:00:00      1
Name: race_date, Length: 205, dtype: int64

In [8]:
df['breed'].value_counts()

TB    4370
Name: breed, dtype: int64

In [9]:
df['name'].value_counts()

Primordia               9
Cape Zavata             8
Foxy Phill              8
Erie Breeze             8
Buckden House           8
                       ..
dq-Too Many Egbert's    1
Nezwaah (GB)            1
Bangla Dancer           1
Uncle Shaggy            1
My Cantata              1
Name: name, Length: 1600, dtype: int64

In [10]:
df['weight'].value_counts()

119    1119
121    1064
118     587
120     417
0       317
117     237
123     198
116     162
115     100
114      83
122      44
124      15
113      13
126      12
112       1
109       1
Name: weight, dtype: int64

In [11]:
df['age'].value_counts()

3     1181
4     1023
2      674
5      562
6      331
0      317
7      208
8       44
9       23
10       4
11       3
Name: age, dtype: int64

In [12]:
df['sex'].value_counts()

G    1824
F    1060
M     853
C     184
H     111
R      21
Name: sex, dtype: int64

In [13]:
df['meds_and_equip'].value_counts()

Lb      1709
L       1533
Lbf      243
Lf       213
b        131
Ly        14
Lv        13
y         10
bf         8
Lby        6
f          5
Lvf        5
Lfy        5
by         4
v          4
Lbfy       1
bfy        1
Lbfr       1
Lbr        1
Name: meds_and_equip, dtype: int64

In [14]:
df['jockey'].value_counts()

E R Da Silva     285
R M Hernandez    284
L   Contreras    258
G   Boulanger    249
D   Moran        240
                ... 
W T Buick          1
C P DeCarlo        1
O   Murphy         1
P   Morales        1
H F Newell         1
Name: jockey, Length: 86, dtype: int64

In [15]:
df['dollar_odds'].value_counts()

0.00      319
2.65       26
3.00       25
2.80       24
2.20       22
         ... 
79.35       1
158.15      1
19.20       1
76.35       1
25.40       1
Name: dollar_odds, Length: 1212, dtype: int64

In [16]:
df['program_number'].value_counts()

5.0     446
2.0     443
1.0     436
4.0     435
3.0     433
6.0     430
7.0     383
8.0     323
9.0     240
10.0    161
11.0    121
12.0     90
13.0     55
14.0     42
15.0     10
16.0      5
Name: program_number, dtype: int64

In [17]:
df['post_position'].value_counts()

1     461
2     459
3     457
4     455
5     454
6     448
7     383
0     317
8     313
9     228
10    144
11    107
12     78
13     40
14     26
Name: post_position, dtype: int64

In [18]:
df['claim_price'].value_counts()

0        1859
9500      475
20000     270
10000     264
40000     245
7500      172
19000     135
8000      121
6000      110
37500     105
12500      87
25000      85
11500      82
32000      67
16000      64
15000      54
6250       45
30000      33
60000      30
62500      20
23500      20
75000      12
50000       9
47500       4
72500       1
80000       1
Name: claim_price, dtype: int64

In [19]:
df['start_position'].value_counts()

1.0     464
3.0     458
4.0     456
2.0     451
5.0     438
6.0     432
0.0     409
7.0     373
8.0     300
9.0     211
10.0    139
11.0    102
12.0     72
13.0     39
14.0     26
Name: start_position, dtype: int64

In [20]:
df['finish_position'].value_counts()

3.0     470
2.0     466
1.0     464
4.0     457
5.0     454
6.0     444
7.0     383
0.0     317
8.0     305
9.0     227
10.0    139
11.0    104
12.0     73
13.0     41
14.0     26
Name: finish_position, dtype: int64

In [21]:
df['finish_time'].value_counts()

0.0    4053
Name: finish_time, dtype: int64

In [22]:
df['speed_rating'].value_counts()

68.0     119
80.0     117
72.0     113
69.0     112
75.0     111
        ... 
22.0       1
19.0       1
17.0       1
27.0       1
107.0      1
Name: speed_rating, Length: 118, dtype: int64

In [23]:
df['trainer'].value_counts()

Casse Mark E          158
Attard Kevin          106
De Paulo Michael P     98
McKnight Norman        94
Doyle Michael J        88
                     ... 
Benson Liam Daniel      1
Connelly Teresa         1
Hickst Waldemar         1
Concessi Armand         1
Smith Andrew            1
Name: trainer, Length: 285, dtype: int64

In [24]:
df['owner'].value_counts()

Colebrook Farms                                 62
Bruno Schickedanz                               58
Joey Gee Thoroughbreds                          47
Tucci Stables                                   45
Chiefswood Stable                               45
                                                ..
Silver King Farm                                 1
Sheikh Ahmed Al Maktoum Lessee and Godolphin     1
Golden Eagle Farm                                1
Ramsey, Kenneth L. and Sarah K.                  1
JMC's Racing Stable                              1
Name: owner, Length: 864, dtype: int64

In [25]:
df['win_payoff'].value_counts()

0.0     3906
5.5        9
6.5        9
4.7        8
5.7        8
        ... 
46.0       1
44.8       1
35.8       1
14.0       1
19.5       1
Name: win_payoff, Length: 194, dtype: int64

In [26]:
df['place_payoff'].value_counts()

0.0     3440
3.0       29
2.9       29
3.6       28
3.3       26
        ... 
14.8       1
25.8       1
22.3       1
14.7       1
27.7       1
Name: place_payoff, Length: 157, dtype: int64

In [27]:
df['show_payoff'].value_counts()

0.0     2994
2.1       83
2.9       61
2.8       59
2.6       57
        ... 
11.1       1
12.9       1
19.8       1
14.6       1
44.9       1
Name: show_payoff, Length: 117, dtype: int64

In [28]:
df['weather'].value_counts()

Cloudy     3145
Showery     544
Clear       425
Rainy       231
Snowing      25
Name: weather, dtype: int64

In [29]:
df['distance'].value_counts()

850.0     1083
600.0      920
700.0      788
650.0      525
500.0      386
900.0      343
800.0      152
550.0       78
1000.0      39
1200.0      25
1400.0      14
1500.0       9
1300.0       8
Name: distance, dtype: int64

In [30]:
df['race_type'].value_counts()

Claiming                       1756
Maiden Claiming                 797
Allowance                       600
Maiden Special Weight           424
Stakes                          365
Allowance Optional Claiming     188
Maiden Optional Claiming        140
Starters Allowance               85
Optional Claiming                15
Name: race_type, dtype: int64

In [31]:
df['track_conditions'].value_counts()

FT    3771
GD     340
FM     252
YL       7
Name: track_conditions, dtype: int64

In [11]:
df = df.drop(['track', 'card_id', 'breed', 'finish_time'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4370 entries, 0 to 4369
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   race_date         4370 non-null   object 
 1   name              4370 non-null   object 
 2   weight            4370 non-null   int64  
 3   age               4370 non-null   int64  
 4   sex               4053 non-null   object 
 5   meds_and_equip    3907 non-null   object 
 6   jockey            4053 non-null   object 
 7   dollar_odds       4370 non-null   float64
 8   program_number    4053 non-null   float64
 9   post_position     4370 non-null   int64  
 10  claim_price       4370 non-null   int64  
 11  start_position    4370 non-null   float64
 12  finish_position   4370 non-null   float64
 13  speed_rating      4053 non-null   float64
 14  trainer           4053 non-null   object 
 15  owner             4053 non-null   object 
 16  win_payoff        4370 non-null   float64


In [12]:
#Upon review there are numerous records with missing values, only including basic information making them unsuitable for our model
df.dropna(subset = ["trainer"], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4053 entries, 0 to 4369
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   race_date         4053 non-null   object 
 1   name              4053 non-null   object 
 2   weight            4053 non-null   int64  
 3   age               4053 non-null   int64  
 4   sex               4053 non-null   object 
 5   meds_and_equip    3907 non-null   object 
 6   jockey            4053 non-null   object 
 7   dollar_odds       4053 non-null   float64
 8   program_number    4053 non-null   float64
 9   post_position     4053 non-null   int64  
 10  claim_price       4053 non-null   int64  
 11  start_position    4053 non-null   float64
 12  finish_position   4053 non-null   float64
 13  speed_rating      4053 non-null   float64
 14  trainer           4053 non-null   object 
 15  owner             4053 non-null   object 
 16  win_payoff        4053 non-null   float64


In [13]:
'''Upon review of the 'meds and equipment' feature, there are some missing values.  Looked up the codes from woodbine's horse program
and 'L' represents no equipment.  Changed all Nulls to L'''
df['meds_and_equip'].fillna(value='L')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4053 entries, 0 to 4369
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   race_date         4053 non-null   object 
 1   name              4053 non-null   object 
 2   weight            4053 non-null   int64  
 3   age               4053 non-null   int64  
 4   sex               4053 non-null   object 
 5   meds_and_equip    3907 non-null   object 
 6   jockey            4053 non-null   object 
 7   dollar_odds       4053 non-null   float64
 8   program_number    4053 non-null   float64
 9   post_position     4053 non-null   int64  
 10  claim_price       4053 non-null   int64  
 11  start_position    4053 non-null   float64
 12  finish_position   4053 non-null   float64
 13  speed_rating      4053 non-null   float64
 14  trainer           4053 non-null   object 
 15  owner             4053 non-null   object 
 16  win_payoff        4053 non-null   float64


In [14]:
df.head()

,race_date,name,weight,age,sex,meds_and_equip,jockey,dollar_odds,program_number,post_position,...,speed_rating,trainer,owner,win_payoff,place_payoff,show_payoff,weather,distance,race_type,track_conditions
0,2015-06-20 00:00:00,Madam Premier,119,4,M,Lb,S Ryan,7.20,1.0,1,...,44.0,O'Keefe Thomas,Emily Anne Walker,0.0,0.0,0.0,Cloudy,900.0,Claiming,FT
1,2015-07-19 00:00:00,Notion Sonet,118,3,C,L,L Contreras,12.10,4.0,4,...,96.0,Carroll Josie,Ivan Dalos,0.0,0.0,0.0,Cloudy,650.0,Allowance,FT
2,2015-07-25 00:00:00,Big Red Bugsy,121,5,G,L,S R Bahen,24.75,6.0,6,...,83.0,Fournier Mark,"Carruthers, Carruthers, Carruthers, Carruthers...",0.0,0.0,0.0,Cloudy,700.0,Claiming,FT
3,2015-08-14 00:00:00,Lillz Pearl,121,5,G,Lb,J McAleney,3.80,3.0,3,...,88.0,Singh Ricky,Ricky Singh,0.0,5.7,3.3,Clear,500.0,Allowance,FT
4,2015-08-29 00:00:00,Sweet Rose,121,4,F,NaN,L Contreras,2.95,3.0,3,...,86.0,Attard Sid C,Stronach Stables,0.0,0.0,3.1,Cloudy,500.0,Allowance,FT


In [15]:
#Created my features and label
Y = df[['finish_position']]
X = df.drop(['finish_position'], axis=1)

In [16]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4053 entries, 0 to 4369
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   race_date         4053 non-null   object 
 1   name              4053 non-null   object 
 2   weight            4053 non-null   int64  
 3   age               4053 non-null   int64  
 4   sex               4053 non-null   object 
 5   meds_and_equip    3907 non-null   object 
 6   jockey            4053 non-null   object 
 7   dollar_odds       4053 non-null   float64
 8   program_number    4053 non-null   float64
 9   post_position     4053 non-null   int64  
 10  claim_price       4053 non-null   int64  
 11  start_position    4053 non-null   float64
 12  speed_rating      4053 non-null   float64
 13  trainer           4053 non-null   object 
 14  owner             4053 non-null   object 
 15  win_payoff        4053 non-null   float64
 16  place_payoff      4053 non-null   float64


In [17]:
encoder = LabelEncoder()
encoder.fit(X['race_date'])
X['race_date'] = encoder.transform(X['race_date'])

In [18]:
#Creating dummies for categorical names
X_dummies = pd.get_dummies(X[['name']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('name', axis=1, inplace=True)

In [19]:
#Creating dummies for categorical sex
X_dummies = pd.get_dummies(X[['sex']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('sex', axis=1, inplace=True)

In [20]:
#Creating dummies for meds and equipment
X_dummies = pd.get_dummies(X[['meds_and_equip']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('meds_and_equip', axis=1, inplace=True)

In [21]:
#Creating dummies for jockey
X_dummies = pd.get_dummies(X[['jockey']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('jockey', axis=1, inplace=True)

In [22]:
#Creating dummies for trainer
X_dummies = pd.get_dummies(X[['trainer']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('trainer', axis=1, inplace=True)

In [23]:
#Creating dummies for owner
X_dummies = pd.get_dummies(X[['owner']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('owner', axis=1, inplace=True)

In [24]:
#Creating dummies for weather
X_dummies = pd.get_dummies(X[['weather']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('weather', axis=1, inplace=True)

In [25]:
#Creating dummies for race type
X_dummies = pd.get_dummies(X[['race_type']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('race_type', axis=1, inplace=True)

In [26]:
#Creating dummies for track conditions
X_dummies = pd.get_dummies(X[['track_conditions']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('track_conditions', axis=1, inplace=True)

In [27]:
X.head()

,race_date,weight,age,dollar_odds,program_number,post_position,claim_price,start_position,speed_rating,win_payoff,...,race_type_Maiden Claiming,race_type_Maiden Optional Claiming,race_type_Maiden Special Weight,race_type_Optional Claiming,race_type_Stakes,race_type_Starters Allowance,track_conditions_FM,track_conditions_FT,track_conditions_GD,track_conditions_YL
0,0,119,4,7.20,1.0,1,9500,5.0,44.0,0.0,...,0,0,0,0,0,0,0,1,0,0
1,1,118,3,12.10,4.0,4,0,10.0,96.0,0.0,...,0,0,0,0,0,0,0,1,0,0
2,2,121,5,24.75,6.0,6,20000,10.0,83.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3,3,121,5,3.80,3.0,3,0,7.0,88.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,4,121,4,2.95,3.0,3,0,4.0,86.0,0.0,...,0,0,0,0,0,0,0,1,0,0


In [28]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4053 entries, 0 to 4369
Columns: 2855 entries, race_date to track_conditions_YL
dtypes: float64(8), int32(1), int64(4), uint8(2842)
memory usage: 11.4 MB


In [29]:
#Merged all the non-winning horses to a single category, we will be predicting either the winner of the race or not the winner
Y['finish_position'] = np.where(Y['finish_position'] > 1, 2, Y['finish_position'])
Y.head()

C:\Users\Mike\AppData\Local\Temp/ipykernel_15424/3521753826.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['finish_position'] = np.where(Y['finish_position'] > 1, 2, Y['finish_position'])


,finish_position
0,2.0
1,2.0
2,2.0
3,2.0
4,2.0


In [30]:
Y = Y.astype(str)
Y_dummies = pd.get_dummies(Y[['finish_position']])
Y = pd.concat([Y,Y_dummies], axis=1)
Y.drop('finish_position', axis=1, inplace=True)

In [31]:
Y.head()

,finish_position_1.0,finish_position_2.0
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


In [32]:
#The next step is to standardize our data (changing the values to between 0 and 1, since our data is sparse)
from sklearn.preprocessing import StandardScaler
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)

In [33]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(X,Y,test_size = 0.2, random_state = 0)

In [34]:
model = keras.Sequential(
    [
        layers.Dense(1400, activation='sigmoid', name='layer1'),
        layers.Dense(700, activation='relu', name='layer2'),
        layers.Dense(200, activation='relu', name='layer3'),
        layers.Dense(20, activation='relu', name='layer4'),
        layers.Dense(2, activation='softmax', name='output1'),
    ]
)

In [35]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True, name="Adam")

In [36]:
model.compile(loss = 'categorical_crossentropy' , optimizer = optimizer , metrics = ['accuracy'] )

In [37]:
#model.summary() # model architecture
checkpointer = ModelCheckpoint(filepath="weights.hdf5", monitor = 'val_accuracy', verbose=1, save_best_only=True) # save model's best weights

history = model.fit(
                    train_x, 
                    train_y, 
                    epochs=100,  
                    callbacks = [checkpointer], 
                    validation_data = (test_x, test_y)
                    )

model_name = "horserace.h5"
model.save(model_name)

Epoch 1/100
102/102 [==============================] - 2s 6ms/step - loss: 0.3560 - accuracy: 0.8871 - val_loss: 0.3693 - val_accuracy: 0.8792

Epoch 00001: val_accuracy improved from -inf to 0.87916, saving model to weights.hdf5
Epoch 2/100
102/102 [==============================] - 0s 4ms/step - loss: 0.3554 - accuracy: 0.8871 - val_loss: 0.3679 - val_accuracy: 0.8792

Epoch 00002: val_accuracy did not improve from 0.87916
Epoch 3/100
102/102 [==============================] - 0s 4ms/step - loss: 0.3511 - accuracy: 0.8871 - val_loss: 0.3740 - val_accuracy: 0.8792

Epoch 00003: val_accuracy did not improve from 0.87916
Epoch 4/100
102/102 [==============================] - 0s 4ms/step - loss: 0.3514 - accuracy: 0.8871 - val_loss: 0.3678 - val_accuracy: 0.8792

Epoch 00004: val_accuracy did not improve from 0.87916
Epoch 5/100
102/102 [==============================] - 0s 4ms/step - loss: 0.3507 - accuracy: 0.8871 - val_loss: 0.3659 - val_accuracy: 0.8792

Epoch 00005: val_accuracy did

In [ ]:
################## Regressor ###########################################

In [64]:
# Checking to see how accurate a regressor we can create by using speed_rating as our target
Y = df[['speed_rating']]
X = df.drop(['speed_rating'], axis=1)

#Recleaning the data
encoder = LabelEncoder()
encoder.fit(X['race_date'])
X['race_date'] = encoder.transform(X['race_date'])

X_dummies = pd.get_dummies(X[['name']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('name', axis=1, inplace=True)

#Creating dummies for categorical sex
X_dummies = pd.get_dummies(X[['sex']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('sex', axis=1, inplace=True)

#Creating dummies for meds and equipment
X_dummies = pd.get_dummies(X[['meds_and_equip']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('meds_and_equip', axis=1, inplace=True)

#Creating dummies for jockey
X_dummies = pd.get_dummies(X[['jockey']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('jockey', axis=1, inplace=True)

#Creating dummies for trainer
X_dummies = pd.get_dummies(X[['trainer']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('trainer', axis=1, inplace=True)

#Creating dummies for owner
X_dummies = pd.get_dummies(X[['owner']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('owner', axis=1, inplace=True)

#Creating dummies for weather
X_dummies = pd.get_dummies(X[['weather']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('weather', axis=1, inplace=True)

#Creating dummies for race type
X_dummies = pd.get_dummies(X[['race_type']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('race_type', axis=1, inplace=True)

#Creating dummies for track conditions
X_dummies = pd.get_dummies(X[['track_conditions']])
X = pd.concat([X,X_dummies], axis=1)
X.drop('track_conditions', axis=1, inplace=True)

X.head()

,race_date,weight,age,dollar_odds,program_number,post_position,claim_price,start_position,finish_position,win_payoff,...,race_type_Maiden Claiming,race_type_Maiden Optional Claiming,race_type_Maiden Special Weight,race_type_Optional Claiming,race_type_Stakes,race_type_Starters Allowance,track_conditions_FM,track_conditions_FT,track_conditions_GD,track_conditions_YL
0,0,119,4,7.20,1.0,1,9500,5.0,5.0,0.0,...,0,0,0,0,0,0,0,1,0,0
1,1,118,3,12.10,4.0,4,0,10.0,4.0,0.0,...,0,0,0,0,0,0,0,1,0,0
2,2,121,5,24.75,6.0,6,20000,10.0,5.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3,3,121,5,3.80,3.0,3,0,7.0,2.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,4,121,4,2.95,3.0,3,0,4.0,3.0,0.0,...,0,0,0,0,0,0,0,1,0,0


In [65]:
#The next step is to standardize our data (changing the values to between 0 and 1, since our data is sparse)
from sklearn.preprocessing import StandardScaler
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)

In [66]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,Y,test_size = 0.2, random_state = 0)

In [67]:
my_regressor = Sequential()
my_regressor.add(LSTM(units = 30, return_sequences = True, 
                      input_shape = (x_train.shape[1], 1)))
my_regressor.add(Dropout(0.2))

my_regressor = keras.Sequential(
    [
        layers.LSTM(1400, activation='sigmoid', name='layer1'),
        layers.Dense(700, activation='relu', name='layer2'),
        layers.Dense(200, activation='relu', name='layer3'),
        layers.Dense(20, activation='relu', name='layer4'),
        layers.Dense(2, activation='softmax', name='output1'),
    ]
)

NameError: name 'Sequential' is not defined

In [ ]:
################################ ML Models ##################################

In [ ]:
prediction_df = pd.DataFrame(model.predict(X_test), columns = ['results'])
prediction_df['podium'] = y_test.reset_index(drop = True)
prediction_df['actual'] = prediction_df.podium.map(lambda x: 1 if x == 1 else 0)
prediction_df.sort_values('results', ascending = True, inplace = True)
prediction_df.reset_index(inplace = True, drop = True)
prediction_df['predicted'] = prediction_df.index
prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

score += precision_score(prediction_df.actual, prediction_df.predicted)
model_score = score / df[df.season == 2019]['round'].unique().max()

In [343]:
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor

params={'fit_intercept': ['True', 'False']}

for fit_intercept in params['fit_intercept']:
    model_params = (fit_intercept)
    model = LinearRegression(fit_intercept = fit_intercept)
    model.fit(X_train, y_train)
            
    comparison_dict['model'].append('linear_regression')
    comparison_dict['params'].append(model_params)
    comparison_dict['score'].append(precision_score(prediction_df.actual, prediction_df.predicted).unique().max())

    
# Random Forest Regressor

params={'criterion': ['mse'],
        'max_features': [0.8, 'auto', None],
        'max_depth': list(np.linspace(5, 55, 26)) + [None]}

for criterion in params['criterion']:
    for max_features in params['max_features']:
        for max_depth in params['max_depth']:
            model_params = (criterion, max_features, max_depth)
            model = RandomForestRegressor(criterion = criterion,
                                          max_features = max_features, max_depth = max_depth, random_state = 1)
            model.fit(X_train, y_train)
            
            model_score = score_regression(model)
            
            comparison_dict['model'].append('random_forest_regressor')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)

            
# Support Vector Machines

params={'gamma': np.logspace(-4, -1, 10),
        'C': np.logspace(-2, 1, 10),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid']} 

for gamma in params['gamma']:
    for c in params['C']:
        for kernel in params['kernel']:
            model_params = (gamma, c, kernel)
            model = svm.SVR(gamma = gamma, C = c, kernel = kernel)
            model.fit(X_train, y_train)
            
            model_score = score_regression(model)
            
            comparison_dict['model'].append('svm_regressor')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)

            
# Neural network

params={'hidden_layer_sizes': [(1400,700,200,20), (1000,500,100,10)], 
        'activation': ['identity', 'relu','logistic', 'tanh',], 
        'solver': ['lbfgs','sgd', 'adam'], 
        'alpha': np.logspace(-4,1,20)} 

for hidden_layer_sizes in params['hidden_layer_sizes']:
    for activation in params['activation']:
        for solver in params['solver']:
            for alpha in params['alpha']:
                model_params = (hidden_layer_sizes, activation, solver, alpha )
                model = MLPRegressor(hidden_layer_sizes = hidden_layer_sizes,
                                      activation = activation, solver = solver, alpha = alpha, random_state = 1)
                model.fit(X_train, y_train)

                model_score = score_regression(model)

                comparison_dict['model'].append('nn_regressor')
                comparison_dict['params'].append(model_params)
                comparison_dict['score'].append(model_score)
  
#print best models  
pd.DataFrame(comparison_dict).groupby('model')['score'].max()  

NameError: name 'comparison_dict' is not defined